In [ ]:
import mujoco
import mujoco.viewer
from testing_env import TestingEnv
import numpy as np
import os
import matplotlib.pyplot as plt

# model = mujoco.MjModel.from_xml_path("assets/env_test_minh.xml")
# data = mujoco.MjData(model)

model_path = os.path.join(os.getcwd(),"assets/env_test_minh.xml")
test_env = TestingEnv(xml_file=model_path)
model, data = test_env.model, test_env.data

vel_hist = []
angle_hist = []

with mujoco.viewer.launch_passive(model, data) as viewer:
	i = 0
	
	# Select the camera
	CAMERA_NAME = "camera"
	viewer.cam.type = mujoco.mjtCamera.mjCAMERA_FIXED
	viewer.cam.fixedcamid = model.camera(CAMERA_NAME).id

	# enable viewer options:
	viewer.opt.frame = mujoco.mjtFrame.mjFRAME_BODY
	viewer.opt.flags[mujoco.mjtVisFlag.mjVIS_JOINT] = True

	while viewer.is_running():
		i+=1

		# --- control simulation
		ctrl = np.zeros(model.nu)
		ctrl[0] = 0.05
		# ctrl[1] = 0.05
		# ctrl[2] = np.sin(i/10000)
		ctrl[2] = 0.005

		# --- simulation
		# mujoco.mj_step(model, data)
		test_env.step(ctrl)
		viewer.sync()
		if (i % 100 == 0):
			# print(test_env.rot_matrix)
			agent_pose, goal_pose = test_env._get_obs()
			x, y, theta, x_dot, y_dot, theta_dot = agent_pose
			# print(f"Current robot angle at time {i:7d}: {theta/np.pi*180:6.2f}", end="\r")
			print(f"Time {i:10d} - Robot position ({x:5.2f},{y:5.2f}) | Goal position {goal_pose}")
			lin_vel = np.sqrt(x_dot**2 + y_dot**2)*100
			angle_hist.append(theta)
			vel_hist.append(lin_vel)
			# print(f"Current robot linear velocities at time {i:7d}: {lin_vel:10.5}", end="\r")

In [ ]:
goal_id = model.body("goal").id
pos = data.xpos[goal_id]
pos 

In [ ]:
figure = plt.figure(figsize=(16,9))
plt.plot(angle_hist, vel_hist)

In [ ]:
theta_range = np.linspace(0, 2*np.pi, num=100)
action = np.array([1, 0, 0], dtype=np.float32)
action_mag_hist = np.zeros(len(theta_range))

for idx, theta in enumerate(theta_range):
    rot_matrix = np.array([[np.cos(theta), -np.sin(theta)],
                        [np.sin(theta), np.cos(theta)]], dtype=np.float32)
    action_transformed = action
    action_transformed[:2] = rot_matrix @ action[:2]
    print(action_transformed)
    action_mag_hist[idx] = np.linalg.norm(action_transformed[:2])




In [ ]:
action_mag_hist